In [ ]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir data
!cp '/content/drive/My Drive/pred-analit/GC-Predictive-Analytics-worksheet - Sheet1 - week 2.csv' data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
df = pd.read_csv('data/GC-Predictive-Analytics-worksheet - Sheet1 - week 2.csv')
df.drop([6, 7, 8])

,cluster,Gender,Age,Ethnicity,Income,Work hours,Health condition,Education,Motivation,Attitude,Intention,Ownership,Unnamed: 12,1,ER,2,ER.1,3,ER.2,4,ER.3,5,ER.4,6,ER.5,7,ER.6,8,ER.7,9,ER.8,10,ER.9,11,ER.10,12,ER.11,13,ER.12,14,ER.13,15,ER.14,16 Recommendation,Actual ER,17 Recommendation,Estimated ER
0,0.0,Female,44.0,Caucasian,"150,000-199,999",50+ hrs/week,Good,4-year college degree,0.265623,0.526590,0.507169,0.559363,NaN,Phy,0.166667,diet,0.714286,diet,0.428571,diet,0.571429,diet,0.750,diet,0.428571,diet,0.857143,diet,1.0,diet,1.0,diet,0.285714,diet,0.571429,diet,0.5,Phy,0.0,Phy,0.000000,Phy,0.000000,Phy,0.0,NaN,NaN
1,1.0,Female,37.0,Asian,"100,000-149,999",16-35 hrs/week,Excellent,2-year college degree,0.386128,0.896973,0.736022,0.924759,NaN,diet,0.000000,diet,0.428571,diet,1.000000,diet,0.857143,diet,0.875,diet,0.857143,diet,1.000000,diet,1.0,diet,1.0,diet,0.857143,diet,0.857143,diet,1.0,diet,1.0,diet,0.857143,diet,0.428571,Phy,0.0,NaN,NaN
2,2.0,Female,60.0,African American,"0-24,999",50+ hrs/week,Excellent,2-year college degree,0.147511,0.593162,0.574159,0.928628,NaN,diet,0.000000,Phy,0.142857,Phy,0.000000,Phy,0.571429,Phy,0.000,Phy,0.000000,Phy,0.000000,Phy,0.0,Phy,0.0,Phy,0.000000,Phy,0.000000,Phy,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,Female,62.0,African American,"0-24,999",1-15 hrs/week,Fair,"Some college, but no degree",0.492041,0.586625,0.645084,0.799052,NaN,Phy,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.0,Female,62.0,African American,"50,000-99,999",1-15 hrs/week,Good,4-year college degree,0.594119,0.563496,0.659809,0.820454,NaN,diet,0.285714,diet,1.000000,Phy,0.470588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,Male,59.0,Caucasian,"50,000-99,999",36-50 hrs/week,Fair,4-year college degree,0.760602,0.541951,0.750184,0.664756,NaN,phy,0.058824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.0,Male,40.0,HIspanic,"0-24,999",50+ hrs/week,Fair,4-year college degree,0.689156,0.647716,0.738081,0.975890,NaN,diet,0.000000,diet,0.000000,diet,0.285714,,0.000000,diet,0.125,diet,0.428571,diet,0.000000,diet,0.0,diet,0.0,diet,0.000000,diet,0.000000,diet,0.0,diet,0.0,diet,0.000000,diet,0.000000,NaN,NaN,NaN,NaN
10,2.0,Male,34.0,Asian,"100,000-149,999",36-50 hrs/week,Good,"Some college, but no degree",0.653392,0.656563,0.732841,0.645978,NaN,Phy,0.000000,diet,0.000000,diet,0.000000,diet,0.000000,diet,0.000,Phy,0.000000,Phy,0.000000,Phy,0.0,Phy,0.0,Phy,0.000000,diet,0.000000,diet,0.0,Phy,0.0,Phy,0.000000,Phy,0.000000,NaN,NaN,NaN,NaN
11,9.0,Female,63.0,Indian/ Asian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phy,0.000000,diet,0.000000,diet,0.000000,diet,0.000000,diet,0.000,Phy,0.000000,Phy,0.000000,Phy,0.0,Phy,0.0,Phy,0.000000,Phy,0.000000,Phy,0.0,Phy,0.0,Phy,0.000000,Phy,0.000000,NaN,NaN,NaN,NaN
12,9.0,Female,49.0,African American,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Phy,0.166667,diet,0.142857,diet,0.285714,diet,0.285714,diet,0.125,Phy,0.142857,Phy,0.000000,Phy,0.0,Phy,0.0,Phy,0.000000,diet,0.000000,diet,0.0,Phy,0.0,Phy,0.000000,Phy,0.000000,NaN,NaN,NaN,NaN


In [ ]:
rec_idxs = [str(i) for i in range(1, 15)]
er_idxs = ['ER']
er_idxs.extend(['ER.{}'.format(i) for i in range(1, 14)])
rec_idxs.append('16 Recommendation')
er_idxs.append('Actual ER')

In [ ]:
rrecs = []
eers = []
iids = []
for i in range(len(df)):
  recs = []
  ers = []
  for rec, er in zip(rec_idxs, er_idxs):
      recomendation = df[rec][i]
      try:
        engagement = float(df[er][i])
      except ValueError:
        engagement = 0.0
      if pd.isna(recomendation):
        continue
      elif recomendation == 'Phy':
        recomendation = 0
      else:
        recomendation = 1
      recs.append(recomendation)
      ers.append(engagement)
  rrecs.append(recs)
  eers.append(ers)
  gender = df.Gender[i]
  hours = df['Work hours'][i]
  hmap = {'1-15 hrs/week': 0.2, '16-35 hrs/week': 0.4, '36-50 hrs/week': 0.6, '50+ hrs/week': 1.0}
  if hours in hmap:
    hours = hmap[hours]
  else:
    hours = 0.0
  if gender == 'Female':
    gender = 0
  else:
    gender = 1
  motivation = df.Motivation[i]	
  if pd.isna(motivation):
    motivation = 0.0
  attitude = df.Attitude[i]
  if pd.isna(attitude):
    attitude = 0
  intention = df.Intention[i]
  if pd.isna(intention):
    intention = 0
  ownership = df.Ownership[i]
  if pd.isna(ownership):
    ownership = 0
  iids.append(dict(age = df.Age[i] / 82,
                   gender = gender,  
                   motivation = motivation,	
                   attitude = attitude, 
                   intention = intention,
                   ownership = ownership,
                   hours = hours))

In [ ]:
X = []
y = []
for i in range(len(rrecs)):
  for j in range(len(rrecs[i]) - 2):
    r1 = rrecs[i][j]
    r2 = rrecs[i][j+1]
    # X.append([eers[i][j], r2 - r1, r1, r2, iids[i]['age']])
    X.append([eers[i][j], r1, r2, r2-r1, iids[i]['age'], iids[i]['motivation']]) #iids[i]['attitude']])
    y.append(eers[i][j+1])

# Method evaluation

### regular features

In [ ]:
models = []
models.append(('Linear', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('Elastic', ElasticNet()))
models.append(('RegTree', RandomForestRegressor()))

# evaluate each model in turn
for name, model in models:
    cv_results = cross_val_score(model, X, y, cv=KFold(n_splits=5, shuffle=True), scoring='r2')
    print("{}: {:.3f} ({})".format(name, cv_results.mean(), cv_results))

Linear: 0.666 ([0.90327221 0.50224958 0.61842319 0.64600184 0.66125944])
Ridge: 0.675 ([0.40078053 0.84365632 0.65191406 0.73002252 0.75044423])
Lasso: -0.036 ([-0.10975348 -0.00022342 -0.00156072 -0.02667716 -0.04385258])
Elastic: -0.080 ([-0.0430803  -0.07851134 -0.06458532 -0.2096477  -0.0040378 ])
RegTree: 0.613 ([0.55179491 0.42228826 0.69582632 0.64096684 0.75535431])


### polynomial features

In [ ]:
poly = PolynomialFeatures(2)
X_ = poly.fit_transform(X)

In [ ]:
models = []
models.append(('Linear', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('Elastic', ElasticNet()))
models.append(('RegTree', RandomForestRegressor()))

# evaluate each model in turn
for name, model in models:
    cv_results = cross_val_score(model, X_, y, cv=KFold(n_splits=5, shuffle=True), scoring='r2')
    print("{}: {:.3f} ({})".format(name, cv_results.mean(), cv_results))

Linear: 0.627 ([0.73910484 0.66962759 0.4955337  0.41026647 0.82264943])
Ridge: 0.678 ([0.90698716 0.7803129  0.63922006 0.74810183 0.31761835])
Lasso: -0.020 ([-0.00017157 -0.02860589 -0.01739532 -0.02699189 -0.02909733])
Elastic: -0.191 ([-0.22384697 -0.01164552 -0.01980247 -0.014259   -0.68786373])
RegTree: 0.604 ([0.49542115 0.86982434 0.74084761 0.13648072 0.7796729 ])


# Weekly prediction

In [ ]:
poly = PolynomialFeatures(2)
X_ = poly.fit_transform(X)
model = RandomForestRegressor()
model.fit(X_, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
for i in [0, 1, 2, 3, 4, 5]:
    r1 = rrecs[i][-1]
    diet_er = model.predict(poly.transform(np.array([eers[i][-1], r1, 1 - r1, 1, iids[i]['age'], iids[i]['motivation']]).reshape(1, -1)))
    phy_er = model.predict(poly.transform(np.array([eers[i][-1], r1, 0 - r1, 0, iids[i]['age'], iids[i]['motivation']]).reshape(1, -1)))
    if diet_er > phy_er:
      print('id: {} recomendation: {} er: {}'.format(i, 'diet', diet_er[0]))
    else:
      print('id: {} recomendation: {} er: {}'.format(i, 'Phy', phy_er[0]))

id: 0 recomendation: diet er: 0.08431863573268021
id: 1 recomendation: diet er: 0.1596043956144153
id: 2 recomendation: diet er: 0.10555512740949251
id: 3 recomendation: diet er: 0.0831632360714181
id: 4 recomendation: diet er: 0.6754255952350999
id: 5 recomendation: Phy er: 0.14192088021444643
